In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import scipy.stats as stats
import pyreadr

# Problem 1

In [3]:
df1 = pd.read_csv('Flu_Shots.txt', sep='\s+', names=['y', 'x1', 'x2', 'x3'])
#df.drop(columns=['Unnamed: 0'], inplace=True)
df1.head()

,y,x1,x2,x3
0,0,59,52,0
1,0,61,55,1
2,1,82,51,0
3,0,51,70,0
4,0,53,70,0


In [4]:
X = df1[['x1', 'x2', 'x3']]
Y = df1['y']
X = sm.add_constant(X)
model = sm.Logit(Y, X).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.330482
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  159
Model:                          Logit   Df Residuals:                      155
Method:                           MLE   Df Model:                            3
Date:                Sun, 18 May 2025   Pseudo R-squ.:                  0.2212
Time:                        11:30:59   Log-Likelihood:                -52.547
converged:                       True   LL-Null:                       -67.470
Covariance Type:            nonrobust   LLR p-value:                 1.486e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.1772      2.982     -0.395      0.693      -7.023       4.668
x1             0.0728      0.

The fitted response function: $\hat{y} = -1.1772 + 0.0728 x_1 - 0.0990 x_2 + 0.4340 x_3$

In [5]:
b0 = model.params['const']
b1 = model.params['x1']
b2 = model.params['x2']
b3 = model.params['x3']

print("exp(b1):", np.exp(b1))
print("exp(b2):", np.exp(b2))
print("exp(b3):", np.exp(b3))

exp(b1): 1.0755025255300643
exp(b2): 0.9057549411897956
exp(b3): 1.5433800567362592


One unit increase in x_1 and x_2 increases the expected value of y by about 1%, while the percent of expected value of y increase by x_3 is about 1,5%

In [6]:
y_head = b0 + b1*55 + b2*60 + b3*1
prob = 1/(1 + np.exp(-y_head))
print("Probability of getting flu shot:", prob)


Probability of getting flu shot: 0.06422196567439759


In [7]:
wald_stat = (b3 / model.bse['x3'])**2
p_value = 1 - stats.chi2.cdf(wald_stat, df=1)

print(f"Wald statistic for X3: {wald_stat:.4f}")
print(f"P-value for Wald test: {p_value:.4f}")


Wald statistic for X3: 0.6917
P-value for Wald test: 0.4056


Hypothesis testing:
$H_0 : \beta_3 = 0$  
$H_1: \beta_3\neq 0$  
Since 0.4056 > 0.05, we fail to reject the null hypothesis at the 5% significance level. There is no strong evidence that $x_3$ has an effect on the dependent variable --> $x_3$ is not statistically significant
# Problem 2

In [8]:
result = pyreadr.read_r('p13.3.rda')
df2 = next(iter(result.values()))
df2

,x,n,r
rownames,,,
1,2500.0,50.0,10.0
2,2700.0,70.0,17.0
3,2900.0,100.0,30.0
4,3100.0,60.0,21.0
5,3300.0,40.0,18.0
6,3500.0,85.0,43.0
7,3700.0,90.0,54.0
8,3900.0,50.0,33.0
9,4100.0,80.0,60.0


In [9]:
def generate_data(df):
    rows = []
    for i, row in df.iterrows():
        # n times y=1
        rows.extend([[row['x'], 1]] * int(row['n']))
        # r times y=0
        rows.extend([[row['x'], 0]] * int(row['r']))
    return pd.DataFrame(rows, columns=['x', 'y'])
df2 = generate_data(df2)
df2

,x,y
0,2500.0,1
1,2500.0,1
2,2500.0,1
3,2500.0,1
4,2500.0,1
...,...,...
1022,4300.0,0
1023,4300.0,0
1024,4300.0,0
1025,4300.0,0


In [10]:
X = df2[['x']]
Y = df2['y']
X = sm.add_constant(X)
model = sm.GLM(Y, X, family=sm.families.Binomial()).fit()
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                 1027
Model:                            GLM   Df Residuals:                     1025
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -630.89
Date:                Sun, 18 May 2025   Deviance:                       1261.8
Time:                        11:30:59   Pearson chi2:                 1.03e+03
No. Iterations:                     4   Pseudo R-squ. (CS):            0.03642
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.3792      0.452      7.472      0.0

The linear regression model is $\hat{y}= 3.3792 - 0.0008x_1$

In [11]:
print("Deviance:", model.deviance)
print("Degrees of freedom:", model.df_resid)
print("Deviance / df:", model.deviance / model.df_resid)

Deviance: 1261.7776903382237
Degrees of freedom: 1025
Deviance / df: 1.2310026247202182


The model fits reasonably well, but there may be some unexplained variability as the division deviance / df = 1.23 is slightly higher than 1

# Problem 3

In [12]:
import numpy as np

# Prepare data
X = df1[['x1', 'x2', 'x3']].values
Y = df1['y'].values
n, k = X.shape
X = np.hstack([np.ones((n, 1)), X])  # Add intercept column

# Gradient ascent for logistic regression
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def log_likelihood(beta, X, Y):
    z = X @ beta
    return np.sum(Y * z - np.log(1 + np.exp(z)))

def gradient(beta, X, Y):
    p = sigmoid(X @ beta)
    return X.T @ (Y - p)

# Initialize
beta = np.zeros(X.shape[1])
learning_rate = 0.001
tol = 1e-6
max_iter = 10000

for i in range(max_iter):
    grad = gradient(beta, X, Y)
    beta_new = beta + learning_rate * grad
    if np.linalg.norm(beta_new - beta, ord=1) < tol:
        break
    beta = beta_new

print("Estimated coefficients (gradient ascent):")
for i, name in enumerate(['Intercept', 'x1', 'x2', 'x3']):
    print(f"{name}: {beta[i]:.4f}")

Estimated coefficients (gradient ascent):
Intercept: 1.0262
x1: 3.9483
x2: -4.5189
x3: 11.9551


C:\Users\ASUS\AppData\Local\Temp\ipykernel_21024\3659158027.py:11: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
